In [7]:
# This assumes that the notebook root is the PyShEx directory
!pip install -e .. --upgrade -q

In [8]:
from rdflib import Graph, Namespace
from pyshex import ShExEvaluator, PrefixLibrary


In [9]:
schema = """
PREFIX ex: <http://ex.example/#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX school: <http://school.example/#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

school:enrolleeAge xsd:integer MinInclusive 13 MaxInclusive 20 

school:Enrollee {
  foaf:age @school:enrolleeAge ;
  ex:hasGuardian IRI {1,2}
}
"""

In [14]:
rdf = """
PREFIX ex: <http://ex.example/#>
PREFIX inst: <http://example.com/users/>
PREFIX school: <http://school.example/#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

inst:Alice foaf:age 13 ;                     
  ex:hasGuardian inst:Person2, inst:Person3 .

inst:Bob foaf:age 15 ;                       
  ex:hasGuardian inst:Person4 .  

inst:Claire foaf:age 12 ;                     
  ex:hasGuardian inst:Person5 .               

inst:Don foaf:age 14 . 

inst:Eric foaf:age 20 ;
  ex:hasGuardian inst:PersonA, inst:PersonB, inst:PersonC .
"""

In [16]:
p = PrefixLibrary(rdf)
for result in ShExEvaluator(rdf=rdf, 
                            schema=schema, 
                            focus=[p.INST.Alice, p.INST.Bob, p.INST.Claire, p.INST.Don, p.INST.Eric], 
                            start=p.SCHOOL.Enrollee).evaluate():
    print(f"{result.focus}: {'Passing' if result.result else 'Failing'}")

http://example.com/users/Alice: Passing
http://example.com/users/Bob: Passing
http://example.com/users/Claire: Failing
http://example.com/users/Don: Failing
http://example.com/users/Eric: Passing
